# 5. Tính toán Học sâu
Trong chương này, chúng tôi sẽ vén tấm màn bí ẩn và đào sâu vào những yếu tố chính của tính toán học sâu; cụ thể là việc xây dựng mô hình, truy cập và khởi tạo tham số, thiết kế các tầng và khối tùy chỉnh, đọc và ghi mô hình lên ổ cứng và cuối cùng là tận dụng GPU nhằm đạt được tốc độ đáng kể. Những hiểu biết này sẽ giúp bạn từ một *người dùng cuối* (**end user**) trở thành một *người dùng thành thạo* (**power user**), cung cấp cho bạn các công cụ cần thiết để gặt hái lợi ích của một thư viện học sâu trưởng thành, đồng thời giữ được sự linh hoạt để lập trình những mô hình phức tạp hơn, bao gồm cả những mô hình mà bạn tự phát minh! Mặc dù chương này không giới thiệu bất cứ mô hình hay tập dữ liệu mới nào, các chương sau về mô hình nâng cao sẽ phụ thuộc rất nhiều vào những kỹ thuật sắp được nhắc đến.

## 5.1. Tầng và Khối
Khi lần đầu giới thiệu về các mạng nơ-ron, ta tập trung vào các mô hình tuyến tính với một đầu ra duy nhất. Như vậy toàn bộ mô hình chỉ chứa một nơ-ron. Lưu ý rằng một nơ-ron đơn lẻ (i) nhận một vài đầu vào; (ii) tạo một đầu ra (vô hướng) tương ứng; và (iii) có một tập các tham số liên quan có thể được cập nhật để tối ưu một hàm mục tiêu nào đó mà ta quan tâm. Sau đó, khi bắt đầu nghĩ về các mạng có nhiều đầu ra, ta tận dụng các phép tính vector để mô tả nguyên một tầng nơ-ron. Cũng giống như các nơ-ron riêng lẻ, các tầng (i) nhận một số đầu vào, (ii) tạo các đầu ra tương ứng, và (iii) được mô tả bằng một tập các tham số có thể điều chỉnh được. Trong hồi quy softmax, bản thân tầng duy nhất ấy chính là một mô hình. Thậm chí đối với các perceptron đa tầng, ta vẫn có thể nghĩ về chúng theo cấu trúc cơ bản này.

Dù bạn có thể nghĩ rằng các nơ-ron, các tầng và các mô hình đã cung cấp đủ sự trừu tượng để bắt tay vào làm việc, hóa ra sẽ là thuận tiện hơn khi ta bàn về các thành phần lớn hơn một tầng riêng lẻ nhưng lại nhỏ hơn toàn bộ mô hình. Ví dụ, kiến trúc ResNet-152, rất phổ biến trong thị giác máy tính, sở hữu hàng trăm tầng. Nó bao gồm các khuôn mẫu nhóm tầng được lặp lại nhiều lần. Việc lập trình từng tầng của một mạng như vậy có thể trở nên tẻ nhạt. Mối quan tâm này không chỉ là trên lý thuyết — các khuôn mẫu thiết kế như vậy rất phổ biến trong thực tế. Kiến trúc ResNet được đề cập ở trên đã giành chiến thắng trong hai cuộc thi thị giác máy tính ImageNet và COCO năm 2015, trong cả bài toán nhận dạng và bài toán phát hiện [He et al., 2016a] và vẫn là một kiến trúc được tin dùng cho nhiều bài toán thị giác. Các kiến trúc tương tự, trong đó các tầng được sắp xếp thành những khuôn mẫu lặp lại, hiện đã trở nên thông dụng ở nhiều lĩnh vực khác, bao gồm cả xử lý ngôn ngữ tự nhiên và xử lý tiếng nói.

Để lập trình các mạng phức tạp này, ta sẽ giới thiệu khái niệm *khối* trong mạng nơ-ron. Một khối có thể mô tả một tầng duy nhất, một mảng đa tầng hoặc toàn bộ một mô hình! Dưới góc nhìn xây dựng phần mềm, một **Block** (Khối) là một lớp. Bất kỳ một lớp con nào của **Block** đều phải định nghĩa phương thức **forward** để chuyển hóa đầu vào thành đầu ra và phải lưu trữ mọi tham số cần thiết. Lưu ý rằng có một vài **Block** sẽ không yêu cầu chứa bất kỳ tham số nào cả! Ngoài ra, một **Block** phải sở hữu một phương thức **backward** cho mục đích tính toán **gradient**. May mắn thay, nhờ sự trợ giúp đắc lực của gói autograd (được giới thiệu trong Section 2) nên khi định nghĩa **Block**, ta chỉ cần quan tâm đến các tham số và hàm **forward**.

![](images/blocks.svg)

Để bắt đầu, ta sẽ xem lại các khối mà ta đã sử dụng để lập trình perceptron đa tầng (Section 4.3). Đoạn mã nguồn sau tạo ra một mạng gồm một tầng ẩn kết nối đầy đủ với 256 nút sử dụng hàm kích hoạt ReLU, theo sau là một *tầng đầu ra* kết nối đầy đủ với 10 nút (không có hàm kích hoạt).

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.1231, -0.1879, -0.1254,  0.1526, -0.2104,  0.0012,  0.0558, -0.0264,
          0.0232, -0.1623],
        [ 0.1281, -0.1477, -0.1358,  0.1242, -0.2951,  0.0239,  0.0534,  0.0285,
          0.1853,  0.0325]], grad_fn=<AddmmBackward>)

Trong ví dụ này, chúng tôi đã xây dựng mô hình của mình bằng cách khởi tạo một `nn.Sequential`, với các lớp theo thứ tự mà chúng sẽ được thực thi dưới dạng các đối số. Tóm lại, `nn.Sequential` định nghĩa một loại Mô-đun đặc biệt, lớp trình bày một khối trong PyTorch. Nó duy trì một danh sách có thứ tự các Mô-đun cấu thành. Lưu ý rằng mỗi lớp trong số hai lớp được kết nối đầy đủ là một thể hiện của lớp Tuyến tính, bản thân nó là một lớp con của Mô-đun. Hàm truyền tiến (chuyển tiếp) cũng rất đơn giản: nó xâu chuỗi từng khối trong danh sách với nhau, chuyển đầu ra của mỗi khối làm đầu vào cho khối tiếp theo. Lưu ý rằng cho đến nay, chúng tôi vẫn đang gọi các mô hình của mình qua `net(X)` để lấy kết quả đầu ra của chúng. Đây thực sự chỉ là cách viết tắt của `net.forward(X)`, một thủ thuật Python mà đạt được thông qua hàm `__call__` của lớp Block.

### 5.1.1. Một Khối Tùy chỉnh
Có lẽ cách dễ nhất để hiểu rõ hơn `nn.Module` hoạt động như thế nào là tự lập trình nó. Trước khi tự lập trình một Block tùy chỉnh, hãy cùng tóm tắt ngắn gọn các chức năng cơ bản mà một Block phải cung cấp:

1. Phương thức forward nhận đối số là dữ liệu đầu vào.
2. Phương thức forward trả về một giá trị đầu ra. Lưu ý rằng đầu ra có thể có kích thước khác với đầu vào. Ví dụ, tầng Dense đầu tiên trong mô hình phía trên nhận đầu vào có kích thước tùy ý nhưng trả về đầu ra có kích thước 256.
3. Tính gradient của đầu ra theo đầu vào bằng phương thức backward, thường thì việc này được thực hiện tự động.
4. Lưu trữ và cung cấp quyền truy cập tới các tham số cần thiết để tiến hành phương thức tính toán forward.
5. Khởi tạo các tham số này khi cần thiết.

Trong đoạn mã dưới đây, chúng ta lập trình từ đầu một Block (Khối) tương đương với một perceptron đa tầng chỉ có một tầng ẩn và 256 nút ẩn, cùng một tầng đầu ra 10 chiều. Lưu ý rằng lớp MLP bên dưới đây kế thừa từ lớp Block. Ta sẽ phụ thuộc nhiều vào các phương thức của lớp cha, và chỉ tự viết phương thức `__init__` và forward.

In [2]:
class MLP(nn.Module):
    # Declare a layer with model parameters. Here, we declare two fully
    # connected layers
    def __init__(self):
        # Call the constructor of the `MLP` parent class `Block` to perform
        # the necessary initialization. In this way, other function arguments
        # can also be specified during class instantiation, such as the model
        # parameters, `params` (to be described later)
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # Hidden layer
        self.out = nn.Linear(256, 10)  # Output layer

    # Define the forward propagation of the model, that is, how to return the
    # required model output based on the input `X`
    def forward(self, X):
        # Note here we use the funtional version of ReLU defined in the
        # nn.functional module.
        return self.out(F.relu(self.hidden(X)))

Để bắt đầu, ta sẽ tập trung vào phương thức forward. Lưu ý rằng nó nhận giá trị đầu vào x, tính toán tầng biểu diễn ẩn và trả về các giá trị logit. Ở cách lập trình MLP này, cả hai tầng trên đều là biến thực thể (instance variables). Để thấy tại sao điều này có lý, tưởng tượng ta khởi tạo hai MLP, net1 và net2, và huấn luyện chúng với dữ liệu khác nhau. Dĩ nhiên là ta mong đợi chúng đại diện cho hai mô hình học khác nhau.

Ta khởi tạo các tầng của MLP trong phương thức `__init__` (hàm khởi tạo) và sau đó gọi các tầng này mỗi khi ta gọi phương thức forward. Hãy chú ý một vài chi tiết quan trọng. Đầu tiên, phương thức `__init__` tùy chỉnh của ta gọi phương thức `__init__` của lớp cha thông qua `super().__init__` để tránh việc viết lại cùng một phần mã nguồn áp dụng cho hầu hết các khối. Chúng ta sau đó khởi tạo hai tầng, gán chúng lần lượt là `self.hidden` và `self.output`. Chú ý rằng trừ khi đang phát triển một toán tử mới, chúng ta không cần lo lắng về lan truyền ngược (phương thức backward) hoặc khởi tạo tham số (phương thức initialize).

In [3]:
net = MLP()
net(X)

tensor([[-0.0605, -0.0267,  0.1868, -0.2107, -0.1201, -0.1622,  0.2295, -0.2193,
          0.0245, -0.0896],
        [-0.1515, -0.0570,  0.0322, -0.0521, -0.0283, -0.0778,  0.1611, -0.1624,
         -0.0517, -0.2121]], grad_fn=<AddmmBackward>)

### 5.1.2. Khối Tuần tự
Bây giờ ta có thể có cái nhìn rõ hơn về cách mà lớp Sequential (Tuần tự) hoạt động. Nhắc lại rằng Sequential được thiết kế để xâu chuỗi các Khối lại với nhau. Để tự xây dựng một lớp MySequential đơn giản, ta chỉ cần định nghĩa hai phương thức chính sau: 1. Phương thức nối thêm nhằm đẩy từng Block một vào trong danh sách. 2. Phương thức forward nhằm truyền một đầu vào qua chuỗi các Blocks (theo thứ tự mà chúng được nối).

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            # Here, `block` is an instance of a `Module` subclass. We save it
            # in the member variable `_modules` of the `Module` class, and its
            # type is OrderedDict
            self._modules[block] = block

    def forward(self, X):
        # OrderedDict guarantees that members will be traversed in the order
        # they were added
        for block in self._modules.values():
            X = block(X)
        return X

Phương thức `__int__` thêm một Block đơn vào từ điển có thứ tự `_modules`. Bạn có thể thắc mắc tại sao mỗi Block sở hữu một thuộc tính `_modules` và tại sao ta sử dụng nó thay vì tự tạo một danh sách Python. Thật ra, ưu điểm chính của `_modules` là trong quá trình khởi tạo trọng số ban đầu của các khối, sẽ tự động tìm các khối con có trọng số cần được khởi tạo trong từ điển này.

Khi phương thức forward của khối `MySequential` được gọi, các Block sẽ được thực thi theo thứ tự mà chúng được thêm vào. Bây giờ ta có thể lập trình lại một MLP sử dụng lớp `MySequential`.

In [6]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.1374,  0.1463, -0.0498, -0.0070, -0.4263, -0.1789,  0.0394,  0.1409,
         -0.0906, -0.1714],
        [-0.1800,  0.2211, -0.0932,  0.1178, -0.2626, -0.1356,  0.1239,  0.2794,
         -0.1232, -0.2464]], grad_fn=<AddmmBackward>)

### 5.1.3. Thực thi Mã trong Phương thức forward
Lớp Sequential giúp việc xây dựng mô hình trở nên dễ hơn, cho phép ta xây dựng các kiến trúc mới mà không cần phải tự định nghĩa một lớp riêng. Tuy nhiên, không phải tất cả mô hình đều có cấu trúc chuỗi xích đơn giản. Khi cần phải linh hoạt hơn, ta vẫn sẽ muốn định nghĩa từng Block theo cách của mình, ví dụ như khi muốn sử dụng luồng điều khiển Python trong lượt truyền xuôi. Hơn nữa, ta cũng có thể muốn thực hiện các phép toán tùy ý thay vì chỉ dựa vào các tầng mạng nơ-ron được định nghĩa từ trước.

Độc giả có thể nhận ra rằng tất cả phép toán trong mạng cho tới giờ đều thao tác trên các giá trị kích hoạt và tham số của mạng. Tuy nhiên, trong một vài trường hợp, ta có thể muốn kết hợp thêm các hằng số. Chúng không phải là kết quả của tầng trước mà cũng không phải là tham số có thể cập nhật được. Ta gọi chúng là tham số không đổi (constant parameter). Ví dụ ta muốn một tầng tính hàm $f(\mathbf{x},\mathbf{w}) = c \cdot \mathbf{w}^\top \mathbf{x}$ , trong đó  $x ,  w$  là tham số, và  $c$  là một hằng số cho trước được giữ nguyên giá trị trong suốt quá trình tối ưu hóa.

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # Random weight parameters that will not compute gradients and
        # therefore keep constant during training
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # Use the created constant parameters, as well as the `relu` and `mm`
        # functions
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # Reuse the fully-connected layer. This is equivalent to sharing
        # parameters with two fully-connected layers
        X = self.linear(X)
        # Control flow
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

Trong mô hình FixedHiddenMLP, ta lập trình một tầng ẩn có trọng số (self.rand_ weight) được khởi tạo ngẫu nhiên và giữ nguyên giá trị về sau. Trọng số này không phải là một tham số mô hình, vì vậy nó không được cập nhật khi sử dụng lan truyền ngược. Sau đó, đầu ra của tầng cố định này được đưa vào tầng kết nối đầy đủ (fully-connected layer).

Lưu ý rằng trước khi trả về giá trị đầu ra, mô hình của ta đã làm điều gì đó bất thường. Ta đã chạy một vòng lặp `while`, lấy vector đầu ra chia cho  $2$  cho đến khi nó thỏa mãn điều kiện `abs(x).sum() > 1`. Cuối cùng, ta gán giá trị đầu ra bằng tổng các phần tử trong x. Theo sự hiểu biết của chúng tôi, không có mạng nơ-ron tiêu chuẩn nào thực hiện phép toán này. Lưu ý rằng phép toán đặc biệt này có thể không hữu ích gì trong các công việc ngoài thực tế. Mục đích của chúng tôi ở đây là chỉ cho độc giả thấy được cách tích hợp một đoạn mã tùy ý vào luồng tính toán của mạng nơ-ron.

In [8]:
net = FixedHiddenMLP()
net(X)

tensor(-0.0411, grad_fn=<SumBackward0>)

Ta có thể kết hợp nhiều cách khác nhau để lắp ráp các Block lại. Trong ví dụ dưới đây, ta lồng các Block với nhau theo nhiều cách sáng tạo.

In [9]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.5175, grad_fn=<SumBackward0>)

### 5.1.4. Biên dịch Mã nguồn¶
Những người đọc có tâm có thể sẽ bắt đầu lo lắng về hiệu năng của một vài đoạn mã trên. Sau cùng thì, chúng ta có rất nhiều thao tác truy cập từ điển, thực thi mã lập trình và rất nhiều thứ “đậm chất Python” khác xuất hiện trong thứ mà lẽ ra nên là một thư viện học sâu hiệu năng cao. Vấn đề của Khóa Trình thông dịch Toàn cục ([Global Interpreter Lock](https://wiki.python.org/moin/GlobalInterpreterLock)) trong Python khá phổ biến. Trong bối cảnh học sâu, ta lo sợ rằng GPU cực kỳ nhanh của ta có thể sẽ phải đợi CPU “rùa bò” chạy xong những dòng lệnh Python trước khi nó có thể nhận tác vụ chạy tiếp theo. Cách tốt nhất để tăng tốc Python là tránh không sử dụng nó.

### 5.1.5. Tóm tắt
* Các tầng trong mạng nơ-ron là các Khối.
* Nhiều tầng có thể cấu thành một Khối.
* Nhiều Khối có thể cấu thành một Khối.
* Một Khối có thể chứa các đoạn mã nguồn.
* Các Khối đảm nhiệm nhiều tác vụ bao gồm khởi tạo tham số và lan truyền ngược.
* Việc gắn kết các tầng và khối một cách tuần tự được đảm nhiệm bởi Khối Sequential.

### 5.1.6. Bài tập
1. Những loại vấn đề nào sẽ xảy ra nếu bạn thay đổi `MySequential` để lưu trữ các khối trong danh sách Python?
2. Hãy lập trình một khối nhận đối số là hai khối khác, ví dụ như `net1` và `net2`, và trả về kết quả là phép nối các giá trị đầu ra của cả hai mạng đó khi thực hiện lượt truyền xuôi.
3. Giả sử bạn muốn nối nhiều thực thể của cùng một mạng với nhau. Hãy lập trình một hàm để tạo ra nhiều thực thể của cùng một mạng và dùng chúng để tạo thành một mạng lớn hơn (các hàm này trong thiết kế phần mềm được gọi là Factory Function).